### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [ ]:
include("jupy.jl")

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

In [ ]:

function solveGoogle(data::DataGoogle, verbose::Bool)
   
    # insert your model here 
    
    

    objVal = 1 # to be replaced by the objective value 

    soluce = Array{Int64}(undef, data.nbProcess) # create an empty table

    for i in 1:data.nbProcess
        soluce[i] = rand(1:data.nbMachines)   # to be replaced by the assignment found by your model 
    end

    return SolutionGoogle(soluce,objVal)
end


### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [ ]:
# enter your instance name here 
donnee = "a1_1"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)
